# Общее описание задачи:

Представьте, что вы работаете в компании, которая разрабатывает мобильные игры. К вам пришел менеджер с рядом задач по исследованию нескольких аспектов мобильного приложения:

В первую очередь, его интересует показатель retention. Напишите функцию для его подсчета.
Помимо этого, в компании провели A/B тестирование наборов акционных предложений. На основе имеющихся данных определите, какой набор можно считать лучшим и на основе каких метрик стоит принять правильное решение.
Предложите метрики для оценки результатов последнего прошедшего тематического события в игре.
 

# EDA

In [11]:
#Импортируем основные библиотеки
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from scipy import stats
from scipy.stats import bootstrap, chi2_contingency
import pingouin as pg

In [12]:
#Выгружаем данные, используем разделитель
reg_df = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-vi-panchenko/shared/problem1-reg_data.csv', sep = ";")
#данные о времени захода пользователей в игру =>
auth_df = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-vi-panchenko/shared/problem1-auth_data.csv', sep = ";")

In [13]:
shape = reg_df.shape[0]
print(f"Количество строк в reg_df: {shape}")

Количество строк в reg_df: 1000000


In [14]:
auth_df.shape[0]
print(f"Количество строк в auth_df: {shape}")

Количество строк в auth_df: 1000000


In [15]:
#Проверка на пропущенные значения в reg_df
missing_values_reg = reg_df.isna().sum()

In [16]:
#Проверка на пропущенные значения в auth_df
missing_values_auth = auth_df.isna().sum()

In [17]:
dtypes_reg = reg_df.dtypes
print(dtypes_reg)

reg_ts    int64
uid       int64
dtype: object


In [18]:
auth_reg = auth_df.dtypes
print(f"Типы данных в auth_df {auth_reg}")

Типы данных в auth_df auth_ts    int64
uid        int64
dtype: object


In [19]:
#Преобразуем колонки reg_ts и auth_ts в колонки с типом данных datetime
reg_df['reg_ts'] = pd.to_datetime(reg_df['reg_ts'], unit='s')
auth_df['auth_ts'] = pd.to_datetime(auth_df['auth_ts'], unit='s')

In [20]:
#Посмотрим на количество уникальныхпользователей в двух датафреймах
unique_users_reg = reg_df['reg_ts'].nunique()
print(f"Число уникальных пользователей в reg_df: {unique_users_reg}")

Число уникальных пользователей в reg_df: 1000000


In [21]:
unique_users_auth = auth_df['auth_ts'].nunique()
print(f"Число уникальных пользователей в auth_df: {unique_users_auth}")

Число уникальных пользователей в auth_df: 9180915


In [22]:
#Посмотрим на максимальную и минимальную дату в датафреймах
min_reg = reg_df['reg_ts'].min()

In [23]:
max_reg = reg_df['reg_ts'].max()

In [24]:
min_auth = auth_df['auth_ts'].min()

In [25]:
max_auth = auth_df['auth_ts'].max()

**Retention** – один из самых важных показателей в компании. Ваша задача – написать функцию, которая будет считать retention игроков (по дням от даты регистрации игрока). Данные лежат в папке shared и имеют следующую структуру:

In [26]:
#Присоединим к датафрейму с авторизациями зарегестрировавшихся пользователей, чтобы сразу отфильтровать тех, кто не авторизовалс
all_df = auth_df.merge(reg_df, how='left', on = 'uid')

In [27]:
#Поменяем порядок столбцов для удобства
all_df = all_df[['uid', 'reg_ts', 'auth_ts']]

Необходимо сформировать **когорты**.
определим основные параметры:

1. Признак формирования когорты: дата регистрации пользователя;
2. Размер когорты: можно выбрать самостоятельно: D-день, M-месяц, Y-год
3. Отчетный период: весь преиод ведения отчетности по пользователям 
4. Анализируемый ключевой показатель: Retention Rate(попробуем рассчитать относительный)


In [28]:
all_df['days_since_reg'] = (all_df['auth_ts'] - all_df['reg_ts']).dt.days

In [29]:
#Добавим когорты на основе первого дня регистрации пользолвателя
all_df['cohort'] = all_df.groupby('uid')['reg_ts'].transform('min').dt.to_period('D')

In [30]:
#Считаем количество уникальных пользователей, группируя по когортам и количеству дней с момента регистрации до авторизации
retention = all_df\
    .groupby(['cohort', 'days_since_reg'])\
    .agg({'uid' : 'nunique'})\
    .reset_index()\
    .rename(columns = {'uid' : 'active_users'})

In [31]:
#Создаем сводную таблицу
pivot_retention = retention.pivot(index = 'cohort', columns = 'days_since_reg', values = 'active_users').fillna(0)

In [32]:
#Идею с div подсказала нейронка!
first_day_users = pivot_retention.iloc[:, 0]

# Делим каждую колонку на количество активных пользователей в первый день и умножаем на 100
pivot_retention_percentage = pivot_retention.div(first_day_users, axis=0) * 100

# Заполняем NaN значениями нулями
pivot_retention_percentage = pivot_retention_percentage.fillna(0).round(1)

# Финальный вариант функции:

In [33]:
#Ver 2.0, в которой пользователь может сам выбирать диапазон дат(с реализацией идеи выбора диапазона дат помогла нейросеть) считает  по дням
def calculate_retention2(reg_df_path, auth_df_path, start_date=None, end_date=None, sep=';'):
    
    '''принимает пути к файлам reg_df и auth_df, возвращает сводную таблицу с подсчетом Retention по сформированным когортам'''
    
    reg_df = pd.read_csv(reg_df_path, sep=sep)
    auth_df = pd.read_csv(auth_df_path, sep=sep)
    
    reg_df['reg_ts'] = pd.to_datetime(reg_df['reg_ts'], unit='s')
    auth_df['auth_ts'] = pd.to_datetime(auth_df['auth_ts'], unit='s')

    if start_date:
        start_date = pd.to_datetime(start_date)
        reg_df = reg_df[reg_df['reg_ts'] >= start_date]
        auth_df = auth_df[auth_df['auth_ts'] >= start_date]

    if end_date:
        end_date = pd.to_datetime(end_date)
        reg_df = reg_df[reg_df['reg_ts'] <= end_date]
        auth_df = auth_df[auth_df['auth_ts'] <= end_date]

    all_df = auth_df.merge(reg_df, how='left', on='uid')
    all_df = all_df[['uid', 'reg_ts', 'auth_ts']]
    all_df['days_since_reg'] = (all_df['auth_ts'] - all_df['reg_ts']).dt.days
    all_df['cohort'] = all_df.groupby('uid')['reg_ts'].transform('min').dt.to_period('M') #можно выбрать период самостоятельно: D-день, M-месяц, Y-год 

    retention = all_df\
        .groupby(['cohort', 'days_since_reg'])\
        .agg({'uid': 'nunique'})\
        .reset_index()\
        .rename(columns={'uid': 'active_users'})

    pivot_retention = retention.pivot(index='cohort', columns='days_since_reg', values='active_users').fillna(0)
    first_day_users = pivot_retention.iloc[:, 0]
    pivot_retention_percentage = pivot_retention.div(first_day_users, axis=0) * 100
    pivot_retention_percentage = pivot_retention_percentage.fillna(0).round(1)
    
    # Настройка отображения сводной таблицы(помогла нейросеть с релаизацией)
    pd.set_option('display.max_rows', None)  # Отображать все строки
    pd.set_option('display.max_columns', None)  # Отображать все столбцы
    pd.set_option('display.width', None)  # Автоматическая ширина


    return pivot_retention_percentage

In [34]:
#Проверка функции
reg_df_path = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-vi-panchenko/shared/problem1-reg_data.csv'
auth_df_path = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-vi-panchenko/shared/problem1-auth_data.csv'

start_date = '2019-01-01'
end_date = '2020-01-01'

retention_percentage = calculate_retention2(reg_df_path, auth_df_path, start_date=start_date, end_date=end_date)
retention_percentage

days_since_reg,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,41.0,42.0,43.0,44.0,45.0,46.0,47.0,48.0,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0,59.0,60.0,61.0,62.0,63.0,64.0,65.0,66.0,67.0,68.0,69.0,70.0,71.0,72.0,73.0,74.0,75.0,76.0,77.0,78.0,79.0,80.0,81.0,82.0,83.0,84.0,85.0,86.0,87.0,88.0,89.0,90.0,91.0,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0,100.0,101.0,102.0,103.0,104.0,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0,114.0,115.0,116.0,117.0,118.0,119.0,120.0,121.0,122.0,123.0,124.0,125.0,126.0,127.0,128.0,129.0,130.0,131.0,132.0,133.0,134.0,135.0,136.0,137.0,138.0,139.0,140.0,141.0,142.0,143.0,144.0,145.0,146.0,147.0,148.0,149.0,150.0,151.0,152.0,153.0,154.0,155.0,156.0,157.0,158.0,159.0,160.0,161.0,162.0,163.0,164.0,165.0,166.0,167.0,168.0,169.0,170.0,171.0,172.0,173.0,174.0,175.0,176.0,177.0,178.0,179.0,180.0,181.0,182.0,183.0,184.0,185.0,186.0,187.0,188.0,189.0,190.0,191.0,192.0,193.0,194.0,195.0,196.0,197.0,198.0,199.0,200.0,201.0,202.0,203.0,204.0,205.0,206.0,207.0,208.0,209.0,210.0,211.0,212.0,213.0,214.0,215.0,216.0,217.0,218.0,219.0,220.0,221.0,222.0,223.0,224.0,225.0,226.0,227.0,228.0,229.0,230.0,231.0,232.0,233.0,234.0,235.0,236.0,237.0,238.0,239.0,240.0,241.0,242.0,243.0,244.0,245.0,246.0,247.0,248.0,249.0,250.0,251.0,252.0,253.0,254.0,255.0,256.0,257.0,258.0,259.0,260.0,261.0,262.0,263.0,264.0,265.0,266.0,267.0,268.0,269.0,270.0,271.0,272.0,273.0,274.0,275.0,276.0,277.0,278.0,279.0,280.0,281.0,282.0,283.0,284.0,285.0,286.0,287.0,288.0,289.0,290.0,291.0,292.0,293.0,294.0,295.0,296.0,297.0,298.0,299.0,300.0,301.0,302.0,303.0,304.0,305.0,306.0,307.0,308.0,309.0,310.0,311.0,312.0,313.0,314.0,315.0,316.0,317.0,318.0,319.0,320.0,321.0,322.0,323.0,324.0,325.0,326.0,327.0,328.0,329.0,330.0,331.0,332.0,333.0,334.0,335.0,336.0,337.0,338.0,339.0,340.0,341.0,342.0,343.0,344.0,345.0,346.0,347.0,348.0,349.0,350.0,351.0,352.0,353.0,354.0,355.0,356.0,357.0,358.0,359.0,360.0,361.0,362.0,363.0,364.0
cohort,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-01,100.0,3.9,4.3,5.1,5.9,6.5,7.6,4.5,5.1,5.1,5.2,5.4,5.1,4.9,4.8,4.7,4.4,4.5,4.3,4.2,3.9,4.0,3.7,3.6,3.6,3.4,3.3,3.2,3.0,3.0,2.8,2.6,2.5,2.6,2.4,2.2,2.3,2.2,1.8,1.8,1.5,1.6,1.5,1.6,1.5,1.5,1.3,1.4,1.3,1.3,1.4,1.3,1.2,1.4,1.3,1.2,1.3,1.3,1.3,1.1,1.4,1.3,1.2,1.2,1.3,1.2,1.2,1.4,1.2,1.3,1.2,1.4,1.3,1.4,1.4,1.3,1.2,1.3,1.4,1.1,1.4,1.3,1.2,1.3,1.3,1.3,1.3,1.3,1.3,1.3,1.2,1.3,1.3,1.1,1.4,1.1,1.4,1.3,1.3,1.3,1.2,1.3,1.3,1.3,1.2,1.3,1.2,1.3,1.3,1.3,1.2,1.3,1.4,1.2,1.2,1.3,1.2,1.3,1.3,1.3,1.3,1.3,1.2,1.3,1.3,1.4,1.1,1.2,1.3,1.2,1.4,1.3,1.3,1.1,1.2,1.3,1.4,1.4,1.2,1.2,1.3,1.4,1.4,1.4,1.2,1.2,1.3,1.2,1.2,1.3,1.4,1.3,1.2,1.4,1.2,1.4,1.3,1.3,1.3,1.2,1.1,1.2,1.3,1.3,1.3,1.2,1.2,1.3,1.2,1.3,1.4,1.2,1.3,1.3,1.4,1.2,1.2,1.3,1.2,1.3,1.3,1.3,1.3,1.1,1.4,1.3,1.4,1.2,1.3,1.3,1.3,1.3,1.3,1.3,1.2,1.3,1.2,1.3,1.3,1.3,1.2,1.2,1.4,1.2,1.4,1.2,1.3,1.3,1.4,1.3,1.2,1.3,1.2,1.3,1.2,1.2,1.4,1.2,1.3,1.3,1.4,1.2,1.3,1.2,1.3,1.4,1.2,1.4,1.3,1.3,1.3,1.3,1.2,1.3,1.3,1.2,1.2,1.2,1.4,1.3,1.4,1.2,1.2,1.3,1.3,1.2,1.2,1.2,1.2,1.3,1.3,1.4,1.3,1.3,1.2,1.3,1.3,1.3,1.2,1.4,1.1,1.3,1.4,1.3,1.3,1.2,1.2,1.4,1.3,1.4,1.2,1.2,1.3,1.2,1.2,1.3,1.2,1.4,1.3,1.2,1.3,1.3,1.2,1.2,1.4,1.3,1.3,1.3,1.3,1.3,1.2,1.2,1.3,1.4,1.4,1.2,1.2,1.4,1.4,1.4,1.4,1.2,1.2,1.4,1.2,1.3,1.3,1.3,1.2,1.3,1.2,1.2,1.3,1.3,1.3,1.2,1.3,1.3,1.2,1.2,1.2,1.3,1.3,1.3,1.2,1.3,1.3,1.3,1.3,1.3,1.2,1.3,1.3,1.3,1.2,1.3,1.2,1.0,1.2,1.0,1.0,1.0,0.9,1.0,0.8,0.8,0.8,0.7,0.7,0.6,0.6,0.6,0.5,0.6,0.5,0.3,0.4,0.4,0.3,0.2,0.2,0.1,0.1,0.1,0.0
2019-02,100.0,4.1,4.4,4.8,5.8,6.2,7.7,4.8,5.2,5.2,5.5,5.2,5.2,4

In [35]:
#Можно сохранить сводную таблицу в эксель формат.
#retention_percentage.to_excel('retention_percentage.xlsx', sheet_name='PivotTable', index=False, header=True)